<a href="https://colab.research.google.com/github/ParkJeongUng/Ung.github.io/blob/master/_notebooks/%EC%95%99%EC%83%81%EB%B8%94_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8_RandomForest_%EC%97%91%EC%8A%A4%ED%8A%B8%EB%9D%BC_%ED%8A%B8%EB%A6%AC_ExtraTreesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블 // 랜덤 포레스트 RandomForest // 엑스트라 트리 ExtraTreesClassifier

## 랜덤 포레스트 // RandomForest
* 랜덤하게 만들어진 여러개의 결정트리가 모인 숲
* 랜덤 포레스트는 각 트리마다 훈련 세트가 다름 --> 각 트리의 훈련 세트를 만들때 전체 훈련세트에서 복원추출을 시행하여 만듬 --> 부트스트랩 샘플
* 랜덤하게 샘플링된 훈련 세트 덕분에 과대적합을 방지해줌

In [ ]:
# 데이터 준비
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [ ]:
wine_input = wine[['alcohol', 'sugar', 'pH']].to_numpy()
wine_target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(wine_input, wine_target, test_size = 0.2, random_state = 42)

In [ ]:
# 랜덤포레스트를 모델로 교차검증 시행하기
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)

score = cross_validate(rf, train_input, train_target ,n_jobs = -1, return_train_score = True) # return_train_score = True --> 훈련 세트에 대한 점수도 반환 --> 과대적합을 확인할때 편함
score

{'fit_time': array([0.68797898, 0.68438673, 0.69149828, 0.68736911, 0.44117284]),
 'score_time': array([0.10288453, 0.1029191 , 0.10267925, 0.10335732, 0.10253263]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [ ]:
# 교차 검증 점수 확인
print(np.mean(score['test_score']), np.mean(score['train_score']))

# 과대적합이 나타남 --> 랜덤포레스트도 결정트리를 사용하는 거기 때문에 가지치기가 필요할 것으로 보임

0.8905151032797809 0.9973541965122431


In [ ]:
# 랜덤포레스트의 특성 중요도 --> feature_importances_
rf.fit(train_input, train_target)
print(rf.feature_importances_)  # 결정트리만 사용했을 때의 feature_importances_ 결과 값 --> [0.12345626   0.86862934   0.0079144]

# 랜덤포레스트는 각 트리마다 전체 특성 개수의 제곱근만큼의 특성을 선택함  ex) 1번 트리 alcohol, sugar / 2번 트리 sugar,pH
# 각 트리마다 특성의 일부만 훈련에 사용하기 때문에 하나의 특성에 과도하게 집중하지 않고 많은 특성을 사용할 수 있음 --> 과대적합을 줄임

[0.23167441 0.50039841 0.26792718]


In [ ]:
# 랜덤포레스트는 전체 훈련세트에서 복원추출 하여 각 트리마다 새로운 훈련세트를 만듬 --> 부트스트랩 샘플
# 복원추출을 했기 때문에 부트스트랩 샘플로 뽑히지 않은 샘플들이 존재 --> OOB 샘플
# 이런 OOB 샘플을 이용해 부트스트랩 샘플로 훈련한 결정트리를 평가 할 수 있음 --> oob_score = True --> OOB 샘플이 검증세트가 되어 교차검증을 하는 느낌

rf = RandomForestClassifier(oob_score = True, random_state = 42, n_jobs = -1)
rf.fit(train_input, train_target)
print(rf.oob_score_)

# cross_validate 로 확인한 점수와 비슷하게 나옴

0.8934000384837406


## 연습

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

wine_input = wine[['alcohol', 'sugar', 'pH']].to_numpy()
wine_target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(wine_input, wine_target, test_size = 0.2, random_state = 42)

score = cross_validate(RandomForestClassifier(random_state = 42), train_input, train_target, n_jobs = -1, return_train_score = True)
print(np.mean(score['test_score']), np.mean(score['train_score']))

rf = RandomForestClassifier(random_state = 42, n_jobs = -1, oob_score = True)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8905151032797809 0.9973541965122431
0.8934000384837406


## 엑스트라 트리
* 부트스트랩 샘플을 사용하지 않는다 --> 알고리즘 훈련 때 전체훈련세트 이용
* 노드를 분할할 때 무작위로 분할한다
* 무작위성 때문에 속도가 빠르다

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]
